# pg_server___buffer_cache 
Analyzing the PostgreSQL Buffer Cache. 

### Backgroun - What is the PostgreSQL Buffer Cache
Buffer cache is the memory area where PostgreSQL stores the most frequently accessed data blocks from the disk. It reduces the need for disk I/O and speeds up the query execution. PostgreSQL uses a least recently used (LRU) algorithm to evict the oldest blocks from the cache when it is full.   
You can control the size of the buffer cache by setting the shared_buffers parameter in the postgresql.conf file. 

### Cells
- Configuration - a connection string
- prerequisites - verify the extension pg_buffercache installed (assumes pg_stat_statements already installed)
- Total buffer size - total size of the buffer cache
- Buffer cache used by each DB - Show how many buffers each DB uses, using the view pg_buffercache. 
- Buffer Cache Hit Ratio
- The Buffer Cache Hit ratio of tables - in a specific database.  

### Resources: 
- [PostgreSQL Internals - Buffer Manager](https://www.interdb.jp/pg/pgsql08.html)


## Configuration
Open a connection to the DB. Use a Connection String stored in a .cfg file

In [1]:
import sqlalchemy
import psycopg2
import pandas as pd
import configparser
import matplotlib.pyplot as plt 
import plotly.figure_factory as ff

# Read from the Config file
config = configparser.ConfigParser() 
config.read_file(open(r'../ipynb.cfg'))

con_str = config.get('con_str', 'PG_AIRBASES') 

engine = sqlalchemy.create_engine(con_str)

# print("Connecting with engine " + str(engine))
try:
    connection = engine.connect()
except (Exception, sqlalchemy.exc.SQLAlchemyError) as error:
    print("Error while connecting to PostgreSQL database:", error)



## Prerequisites
Check whether the extension exists.   

TODO: The cell below doesn't really work. The query returns "false" but the code ignores it. Maybe it returns the string "false" rather than a bit

In [2]:

from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError

sql_command = """
SELECT CASE WHEN COUNT(*) > 0 THEN B'1'::BIT(1) ELSE B'0'::BIT(1) END AS extension_exists
FROM pg_extension
WHERE extname = 'pg_buffercache';
"""
df = pd.read_sql_query(sql_command, connection)

try:
    # Execute the SQL command
   
    result = connection.execute(sql_command).fetchone()

    # print (type(result[0]))
        # Check the result
    if result[0] == '0':
        raise ValueError("Error: The extension pg_buffercache doesn't exist ")
    else: 
        print ("The Extension pg_buffercache exists")
except (SQLAlchemyError, ValueError) as e:
    # Handle any errors or raised exceptions
    raise e


The Extension pg_buffercache exists


## Total Buffer Size
The ```pg_buffercache``` module provides a means for examining what's happening in the shared buffer cache in real time.
The module provides a C function pg_buffercache_pages that returns a set of records, plus a view pg_buffercache that wraps the function for convenient use. https://www.postgresql.org/docs/current/pgbuffercache.html.   

The total size of the buffer cache should be equal to the setting  ```shared_buffers``` ( run the command: ```Show shared_buffers```)

In [3]:
qry_total_buffer_size = """
SELECT 
	count(*) as count_buffers, 
	count(*) * 8 as buffers_size_kb,
  pg_size_pretty(count(*) * 8) as buffer_size_pretty
FROM pg_buffercache;
"""

df = pd.read_sql_query(qry_total_buffer_size, connection)
df

,count_buffers,buffers_size_kb,buffer_size_pretty
0,243066,1944528,1899 kB


## Buffers used by Each DB
Shows how many buffers each db uses (as a number as in KB). It also shows the percentage of each DB buffers usage from total.
Resources: https://tomasz-gintowt.medium.com/postgresql-extensions-pg-buffercache-b38b0dc08000 

TODO: exclude non important databases such as template0, rdsadmin

In [4]:

qry_total_buffer_size = """
WITH total_num_of_buffers AS (
    SELECT count(*)  AS count_buffers
    FROM pg_buffercache
)
SELECT 
    d.datname AS db_name, 
    count(*) AS count_buffers,
    count(*) * 8 AS total_buffers_size_kb, 
    pg_size_pretty(count(*) * 8 * 1024) AS total_buffers_size,
    round((count(*)) * 100.0 / t.count_buffers, 2) AS percentage_of_total
FROM pg_buffercache b
INNER JOIN pg_database d ON b.reldatabase = d.oid
CROSS JOIN total_num_of_buffers t

GROUP BY d.datname, 
	t,count_buffers
ORDER BY d.datname;
"""

df = pd.read_sql_query(qry_total_buffer_size, connection)
table = ff.create_table(df)
table.show()

### Buffers used by each DB - Pie Chart

Todo: Group all the DB that uses a small number of buffers into one logical unit. 

In [5]:
import plotly.express as px

# Assuming you already have df with the SQL query results

# Create a pie chart using Plotly Express
fig = px.pie(df, names='db_name', values='count_buffers', title='Buffer Cache Blcoks')

# Show the pie chart
fig.show()


## Databases Cache Hit Ratio
**Cache Hit Ratio** is a metric that quantifies the efficiency of a cache system. It represents the percentage of requested data or items that are successfully retrieved from the cache (hits) compared to the total number of requests made to the cache (hits plus misses). A higher cache hit ratio indicates that the cache is effectively reducing the need to access the slower, underlying data source, leading to improved system performance.The **Cache Hit Ratio** should be around 99%.  

Notice: The extension pg_stat_statements must be installed to query the table ```pg_stat_database```
- [pg_stat_database](https://www.postgresql.org/docs/current/monitoring-stats.html#MONITORING-PG-STAT-DATABASE-VIEW)
- The changes in the system view in every [PG version](https://pgstats.dev/pg_stat_database)

In [6]:
import plotly.graph_objects as go

qry_database_activity = """
SELECT 
  datid as dbid, 
  datname as db_name, 
  blks_read, 
  blks_hit, 
  round(blks_hit / (blks_read + blks_hit)::numeric, 4) as cache_hit_ratio
FROM pg_stat_database
ORDER BY cache_hit_ratio DESC """ 
df = pd.read_sql_query(qry_database_activity, connection)


# Define a function to set row font color based on cache_hit_ratio
def set_font_color(value):
    if value > 0.95:
        return '<span style="color: green;">{}</span>'.format(value)
    else:
        return '<span style="color: red;">{}</span>'.format(value)

# Apply the font color function to the relevant column (in this case, cache_hit_ratio)
df['dynamic_font_color'] = df['cache_hit_ratio'].apply(set_font_color)


table = ff.create_table(df[['dbid', 'db_name', 'blks_read', 'blks_hit', 'dynamic_font_color']])

table.show()



## Tables (in a single DB) Cache Hit Ratio.
Postgres caches recent queries in memory called the shared buffer cache (shared_buffers in postgresql.conf). The pg_statio_user_tables has as rows representing various stats on each of the (user) tables. The two columns of interest we’ll be looking at are:

* pg_statio_user_tables.heap_blks_read — Number of disk blocks read from a table (ie. missed cache)
* pg_statio_user_tables.heap_blks_hits — Number of buffer hits from this table (ie. cache hit)

TODO: also show the actual table size and the actual size of the table in the buffer now. 

In [7]:
query_ccnnections_metrics = """
SELECT 
  schemaname, 
  pg_class.relname, 
  ROUND(
    CASE 
      WHEN heap_blks_hit + heap_blks_read = 0 
      THEN 0 
      ELSE heap_blks_hit::decimal / (heap_blks_hit + heap_blks_read) 
    END, 
    3
  ) as cache_hit_ratio
FROM 
  pg_statio_user_tables 
  JOIN pg_class ON pg_statio_user_tables.relid = pg_class.oid 
  JOIN pg_namespace ON pg_namespace.oid = pg_class.relnamespace;
"""

df_tables = pd.read_sql_query(query_ccnnections_metrics, connection)
table = ff.create_table(df_tables[['schemaname', 'relname', 'cache_hit_ratio']])

table.show()

## Cache Fragmentation
### TODO - Not ready yet
Resources: 
- [How do you troubleshoot PostgreSQL's buffer cache bloat and fragmentation issues and reclaim wasted space](https://www.linkedin.com/advice/0/how-do-you-troubleshoot-postgresqls-buffer-cache-bloat)   
  

What is **cache fragmentation**?
Cache fragmentation is a situation where the buffer cache contains many blocks that are partially filled or have gaps between them. This can happen when a table or index is subject to frequent insertions, deletions, or updates that change the size or location of the data. Cache fragmentation reduces the effective use of the cache space and increases the disk I/O. You can measure the cache fragmentation by using the ```pg_freespacemap``` extension or the ```pgstattuple``` function.